# Connectors

Connectors enable interactions between different modules in Starsim, particularly between disease models. They allow you to model complex epidemiological scenarios where one disease affects the transmission or progression of another disease. For example, you might want to model how having syphilis increases susceptibility to HIV, or how HIV affects the treatment outcomes of other sexually transmitted infections.

## Basic concepts

A connector inherits from `ss.Connector` and implements custom logic to modify disease parameters or agent states based on the presence of other diseases. Connectors are called during each simulation timestep and can dynamically adjust transmission parameters, susceptibility, or other disease characteristics.

## Simple usage

The most straightforward way to use connectors is to define a custom class that inherits from `ss.Connector` and implements a `step()` method. This method is called during each simulation timestep and can modify disease parameters based on the current state of the simulation.

Here's a simple example showing how HIV and SIS (Susceptible-Infected-Susceptible) diseases can interact:

In [ ]:
import starsim as ss
import numpy as np
import matplotlib.pyplot as plt

class SIS_HIV(ss.Connector):
    """
    Simple connector that modifies HIV susceptibility based on SIS infection status.
    People with SIS infection are protected from HIV (rel_sus = 0),
    while susceptible people have normal HIV susceptibility (rel_sus = 1).
    """
    
    def step(self):
        # Get disease modules
        hiv = self.sim.diseases.hiv
        sis = self.sim.diseases.sis
        
        # Modify HIV relative susceptibility based on SIS infection status
        hiv.rel_sus[:] = 1.0  # Reset to baseline
        hiv.rel_sus[sis.infected] = 0.0  # SIS-infected people cannot acquire HIV

# Create disease models
hiv = ss.HIV(beta=dict(mf=1.0, random=0))
sis = ss.SIS(beta=dict(mf=0.0, random=0.05))

# Configure simulation parameters
kw = dict(
    diseases=[hiv, sis], 
    networks=['mf','random'], 
    n_agents=2000,
    verbose=0
)

# Create simulations with and without connector
s1 = ss.Sim(label='Without connector', **kw)
s2 = ss.Sim(label='With connector', **kw, connectors=SIS_HIV())

# Run simulations in parallel
msim = ss.parallel(s1, s2)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot HIV infections
axes[0].plot(s1.timevec, s1.results.hiv.n_infected, label='Without connector')
axes[0].plot(s2.timevec, s2.results.hiv.n_infected, label='With connector')
axes[0].set_title('HIV infections over time')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Number infected')
axes[0].legend()

# Plot SIS infections
axes[1].plot(s1.timevec, s1.results.sis.n_infected, label='Without connector')
axes[1].plot(s2.timevec, s2.results.sis.n_infected, label='With connector')
axes[1].set_title('SIS infections over time')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Number infected')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Final HIV infections - Without connector: {s1.results.hiv.n_infected[-1]}")
print(f"Final HIV infections - With connector: {s2.results.hiv.n_infected[-1]}")
print(f"Final SIS infections - Without connector: {s1.results.sis.n_infected[-1]}")
print(f"Final SIS infections - With connector: {s2.results.sis.n_infected[-1]}")

In this example, the connector modifies HIV susceptibility based on SIS infection status. You can see that when the connector is active, HIV transmission is significantly reduced because SIS-infected individuals cannot acquire HIV.

## Advanced usage with parameters

For more complex interactions, you can define parameters in your connector class to control the strength and nature of the interactions. Here's an example showing bi-directional interactions between HIV and syphilis:

In [ ]:
class HIV_Syphilis(ss.Connector):
    """
    Advanced connector showing bi-directional interactions between HIV and syphilis.
    Models how HIV affects syphilis susceptibility/transmission and vice versa,
    with different effects based on CD4 count (AIDS progression).
    """
    
    def __init__(self, **kwargs):
        super().__init__()
        self.define_pars(
            label='HIV-Syphilis',
            rel_sus_syph_hiv=2,      # People with HIV are 2x more likely to acquire syphilis
            rel_sus_syph_aids=5,     # People with AIDS are 5x more likely to acquire syphilis
            rel_trans_syph_hiv=1.5,  # People with HIV are 1.5x more likely to transmit syphilis
            rel_trans_syph_aids=3,   # People with AIDS are 3x more likely to transmit syphilis
            rel_sus_hiv_syph=2.7,    # People with syphilis are 2.7x more likely to acquire HIV
            rel_trans_hiv_syph=2.7,  # People with syphilis are 2.7x more likely to transmit HIV
        )
        self.update_pars(**kwargs)
        return

    def step(self):
        """Apply HIV-syphilis interactions based on CD4 count and infection status"""
        diseases = self.sim.diseases
        syph = diseases.syphilis
        hiv = diseases.hiv
        cd4 = self.sim.people.hiv.cd4

        # Reset relative susceptibility and transmission to baseline
        syph.rel_sus[:] = 1.0
        syph.rel_trans[:] = 1.0
        hiv.rel_sus[:] = 1.0
        hiv.rel_trans[:] = 1.0

        # People with HIV are more likely to acquire syphilis
        syph.rel_sus[cd4 < 500] = self.pars.rel_sus_syph_hiv
        syph.rel_sus[cd4 < 200] = self.pars.rel_sus_syph_aids  # AIDS stage

        # People with HIV are more likely to transmit syphilis
        syph.rel_trans[cd4 < 500] = self.pars.rel_trans_syph_hiv
        syph.rel_trans[cd4 < 200] = self.pars.rel_trans_syph_aids

        # People with syphilis are more likely to acquire HIV
        hiv.rel_sus[syph.active] = self.pars.rel_sus_hiv_syph

        # People with syphilis are more likely to transmit HIV
        hiv.rel_trans[syph.active] = self.pars.rel_trans_hiv_syph
        return

# Create disease models
hiv = ss.HIV(beta={'mf': [0.0008, 0.0004]}, init_prev=0.2)
syph = ss.Syphilis(beta={'mf': [0.1, 0.05]}, init_prev=0.05)

# Create network
mf = ss.MFNet()

# Configure simulation parameters
kw = dict(
    diseases=[hiv, syph], 
    networks=mf,
    n_agents=2000,
    verbose=0
)

# Create simulations
s1 = ss.Sim(label='Without connector', **kw)
s2 = ss.Sim(label='With connector', connectors=HIV_Syphilis(), **kw)

# Run simulations
msim = ss.parallel(s1, s2)

# Compare results
print("\\nFinal infection counts:")
print(f"HIV infections - Without connector: {s1.results.hiv.n_infected[-1]}")
print(f"HIV infections - With connector: {s2.results.hiv.n_infected[-1]}")
print(f"Syphilis infections - Without connector: {s1.results.syphilis.n_infected[-1]}")
print(f"Syphilis infections - With connector: {s2.results.syphilis.n_infected[-1]}")

# Plot comparative results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# HIV infections
axes[0].plot(s1.timevec, s1.results.hiv.n_infected, label='Without connector', alpha=0.8)
axes[0].plot(s2.timevec, s2.results.hiv.n_infected, label='With connector', alpha=0.8)
axes[0].set_title('HIV infections over time')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Number infected')
axes[0].legend()

# Syphilis infections
axes[1].plot(s1.timevec, s1.results.syphilis.n_infected, label='Without connector', alpha=0.8)
axes[1].plot(s2.timevec, s2.results.syphilis.n_infected, label='With connector', alpha=0.8)
axes[1].set_title('Syphilis infections over time')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Number infected')
axes[1].legend()

plt.tight_layout()
plt.show()

This more complex example demonstrates several important concepts:

1. **Parameter definition**: Using `define_pars()` to set configurable parameters for the interaction strengths
2. **Bi-directional effects**: Both diseases affect each other's transmission and susceptibility
3. **Disease progression considerations**: Different effects based on HIV progression (CD4 count)
4. **Multiple attributes**: Modifying both relative susceptibility (`rel_sus`) and relative transmission (`rel_trans`)

## Combining connectors with interventions

Connectors can work alongside interventions to model complex treatment scenarios. Here's an example that combines the HIV-syphilis connector with a treatment intervention:

In [ ]:
class Penicillin(ss.Intervention):
    """
    Penicillin treatment intervention for syphilis.
    Also resets HIV transmission/susceptibility parameters when syphilis is cured.
    """
    
    def __init__(self, year=2020, prob=0.8):
        super().__init__()
        self.prob = prob  # Probability of receiving treatment
        self.year = ss.date(year)  # Year treatment becomes available
        return

    def step(self):
        sim = self.sim
        if sim.now >= self.year:
            syphilis = sim.diseases.syphilis

            # Define who is eligible for treatment (currently infected)
            eligible_ids = syphilis.infected.uids
            n_eligible = len(eligible_ids)

            if n_eligible > 0:
                # Determine who receives treatment
                is_treated = np.random.rand(n_eligible) < self.prob
                treat_ids = eligible_ids[is_treated]

                # Cure syphilis
                syphilis.infected[treat_ids] = False
                syphilis.susceptible[treat_ids] = True

                # Reset HIV parameters (removes syphilis co-infection effects)
                sim.diseases.hiv.rel_sus[treat_ids] = 1.0
                sim.diseases.hiv.rel_trans[treat_ids] = 1.0
        return

# Create simulations with different combinations
sims = {}

# Base configuration
base_kw = dict(
    diseases=[ss.HIV(beta={'mf': [0.0008, 0.0004]}, init_prev=0.2),
              ss.Syphilis(beta={'mf': [0.1, 0.05]}, init_prev=0.05)], 
    networks=ss.MFNet(),
    n_agents=2000,
    verbose=0
)

# 1. No connector, no treatment
sims['baseline'] = ss.Sim(label='Baseline (no connector, no treatment)', **base_kw)

# 2. Connector only
sims['connector'] = ss.Sim(label='With connector only', 
                          connectors=HIV_Syphilis(), **base_kw)

# 3. Connector + treatment
sims['both'] = ss.Sim(label='Connector + treatment', 
                     connectors=HIV_Syphilis(), 
                     interventions=Penicillin(year=2020, prob=0.8), **base_kw)

# Run all simulations
msim = ss.parallel(*sims.values())

# Create comparison plot
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# HIV infections
axes[0,0].set_title('HIV infections over time')
for sim in sims.values():
    axes[0,0].plot(sim.timevec, sim.results.hiv.n_infected, label=sim.label, alpha=0.8)
axes[0,0].axvline(2020, color='red', linestyle='--', alpha=0.5, label='Treatment start')
axes[0,0].set_xlabel('Year')
axes[0,0].set_ylabel('Number infected')
axes[0,0].legend()

# Syphilis infections
axes[0,1].set_title('Syphilis infections over time')
for sim in sims.values():
    axes[0,1].plot(sim.timevec, sim.results.syphilis.n_infected, label=sim.label, alpha=0.8)
axes[0,1].axvline(2020, color='red', linestyle='--', alpha=0.5, label='Treatment start')
axes[0,1].set_xlabel('Year')
axes[0,1].set_ylabel('Number infected')
axes[0,1].legend()

# Cumulative HIV infections
axes[1,0].set_title('Cumulative HIV infections')
for sim in sims.values():
    axes[1,0].plot(sim.timevec, sim.results.hiv.cum_infections, label=sim.label, alpha=0.8)
axes[1,0].set_xlabel('Year')
axes[1,0].set_ylabel('Cumulative infections')
axes[1,0].legend()

# Cumulative syphilis infections
axes[1,1].set_title('Cumulative syphilis infections')
for sim in sims.values():
    axes[1,1].plot(sim.timevec, sim.results.syphilis.cum_infections, label=sim.label, alpha=0.8)
axes[1,1].set_xlabel('Year')
axes[1,1].set_ylabel('Cumulative infections')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Print summary statistics
print("\\nFinal results summary:")
print("-" * 60)
for sim in sims.values():
    hiv_final = sim.results.hiv.n_infected[-1]
    syph_final = sim.results.syphilis.n_infected[-1]
    hiv_cum = sim.results.hiv.cum_infections[-1]
    syph_cum = sim.results.syphilis.cum_infections[-1]
    print(f"{sim.label}:")
    print(f"  HIV: {hiv_final:3.0f} current, {hiv_cum:4.0f} cumulative")
    print(f"  Syphilis: {syph_final:3.0f} current, {syph_cum:4.0f} cumulative")
    print()

This example demonstrates how interventions and connectors work together. The treatment intervention not only cures syphilis but also resets the HIV transmission parameters, effectively removing the co-infection effects when syphilis is treated.

## Key concepts and best practices

When creating connectors, keep these important points in mind:

### 1. Connector execution order
Connectors are executed during each timestep after disease transmission but before updating disease states. This means they can influence transmission for the current timestep.

### 2. Resetting parameters
Always reset modified parameters to baseline values before applying new effects. This prevents parameters from accumulating changes over time.

### 3. Parameter types
Connectors commonly modify these disease parameters:
- `rel_sus`: Relative susceptibility (how likely someone is to acquire a disease)
- `rel_trans`: Relative transmission (how likely someone is to transmit a disease)
- Other disease-specific parameters (e.g., viral load, treatment outcomes)

### 4. Accessing disease states
Common ways to identify infected individuals:
- `disease.infected`: Boolean array indicating current infection status
- `disease.active`: Similar to infected but may include different disease stages
- `disease.ti_infected`: Timestep when infection occurred
- Disease-specific attributes (e.g., `hiv.cd4` for CD4 count)

### 5. Performance considerations
- Keep connector logic simple and efficient since it runs every timestep
- Use vectorized operations when possible
- Avoid complex computations that could slow down large simulations

Connectors are a powerful tool for modeling complex disease interactions and can significantly enhance the realism of your epidemiological simulations. They allow you to capture important biological and social factors that influence how diseases interact within populations.